# Capacitated Vehicle Routing with Time Window

## Definizione

Il Capacited Vehicle Routing Problem (CVRP) è un'estensione del VRP. In un problema di CVRP, ogni veicolo ha una determinata capacità e non può caricare più merci della sua capacità. L'obiettivo è trovare il percorso ottimale minimizzando i costi di trasporto, il numero di veicoli usati e la qualità del servizio. Il Capacitated Vehicle Routing Problem with Time Windows è il CVRP con vincoli di tempo. In questa variante il cliente i-esimo può essere servitor solo in un dato periodo di tempo e con un tempo di servizio.

Il CVRPTW è un problema di ottimizzazione combinatoria in cui, dato un grafo G(V,A):
<ul>
  <li>V: insieme dei vertici</li>
  <li>A: insieme degli archi</li>
  <li>Q: capacità dei veicolo</li>
  <li>M: numero massimo di veicoli</li>
</ul>

Si vuole determinare una soluzione ammissibile formata da una famiglia di cluster veicoli / clienti dove, per ogni cluster, la capacità di un veicolo non è superata. Una generica soluzione T è dunque un insieme di cicli hamiltoniano cioè un insieme di percorsi che toccano tutti i nodi di un cluster una sola volta (partendo dal deposito).


### Funzioni di supporto

In [ ]:
import gurobipy as gb
from gurobipy import *
import re
import math
import matplotlib.pyplot as plt
import networkx as nx

# Calcolo della distanza tra due nodi
def distance(node1, node2):
    i = node1["coordinate"]
    j = node2["coordinate"]
    diff = (i[0] - j[0], i[1] - j[1])
    return math.sqrt(diff[0] * diff[0] + diff[1] * diff[1])

# Calcolo del tempo di viaggio tra due nodi
def travel_time(node1, node2):
    return round(distance(node1,node2), 1)

# Calcola della matrice delle distanze: matrice in cui l'elemento (i,j) indica la distanza tra il nodi i-esimo e il nodo j-esimo
def distance_matrix(problem):
    problem["d_matrix"] = {(i,j): distance(problem["nodes"][i], problem["nodes"][j]) for i in range(problem["nodes_num"]) for j in range(problem["nodes_num"]) if i != j}

# Calcola della matrice dei tempi di viaggio: matrice in cui l'elemento (i,j) indica il tempo di viaggio tra il nodi i-esimo e il nodo j-esimo
def travel_matrix(problem):
    problem["t_matrix"] = {(i,j): round(problem["d_matrix"][i, j], 1) for i in range(problem["nodes_num"]) for j in range(problem["nodes_num"]) if i != j}


### Caricamento del Dataset

In [ ]:
#file = "./Dataset/Homberger/C1_2_1.TXT"
file = "../raimo/Desktop/RO_progetto/Dataset/Homberger/C1_2_1.TXT"

nodes_num = 0 # Numero di nodi
nodes = [] # Lista di nodi

f = open(file, "r")
line = f.readline()
line.strip("\n")
name_istance = line

for i in range (3):
    line = f.readline() # Salto 3 linee

line = f.readline()
data = re.findall(r"[-+]?\d.\d+|\d+", line)
num_vh = int(data[0])
capacity = int(data[1])  

for i in range (4):
    line = f.readline() # Salto 4 linee

line = f.readline()
while not line.startswith("EOF"):
    data = re.findall(r"[-+]?\d.\d+|\d+", line)
    coordinate = (int(data[1]), int(data[2]))
    demand = int(data[3])
    rdy_time = int(data[4])
    due_date = int(data[5])
    serv_time = int(data[6])
    nodes.append({"number": nodes_num, "coordinate": coordinate, "demand": demand, "rdy_time": rdy_time, "due_date": due_date, "service_time": serv_time})
    nodes_num += 1
    line = f.readline()
f.close()

problem = {"nodes_num": nodes_num , "nodes": nodes, "name_instance": name_istance, "num_veh": num_vh, "capacity": capacity, "d_matrix": [] , "t_matrix": [] , "routes": []}

distance_matrix(problem)
travel_matrix(problem)

print(nodes_num)


### Struttura dati

### Formulazione

Le variabili utilizzate sono:
- $x_{ij}$: 1 se l'arco $(i,j)$ è attraversato, 0 altrimenti
- $a_{i}$: tempo di inizio del servizio al cliente $i-esimo$
- $u_{i}$: quantità di domanda servita dal veicolo nel percorso dal deposito al nodo

In [ ]:
mod = gb.Model('CVRP')
x = mod.addVars(range(problem["nodes_num"]), range(problem["nodes_num"]), vtype=GRB.BINARY, name="X")
u = mod.addVars(range(problem["nodes_num"]), vtype=GRB.INTEGER, name="U")
a = mod.addVars(range(problem["nodes_num"]), vtype=GRB.INTEGER, name="A")

### Vincoli

- Se il veicolo entra in i, deve anche uscire

\begin{equation}
\sum_{j \in V} x_{ij} = \sum_{j \in V}^n x_{ji} \quad \; \; \forall \;i \; \in \; V
\tag{1}
\end{equation}

- Dal deposito devono uscire non più di M veicoli

\begin{equation}
\sum\limits_{j=1}^n x_{0j} \leq M
\tag{2}
\end{equation}

- Ad ogni cliente deve arrivare un unico veicolo

\begin{equation}
\sum_{j \in V} x_{ij} = 1 \quad \; \; \forall \;i \; \in \; {1, .... , n}
\tag{1}
\end{equation}

- Quando il camion parte dal deposito non ho ancora servito alcuna domanda

\begin{equation}
u_{0} = 0
\tag{4}
\end{equation}

- Quando il camion arriva al cliente i, la domanda totale servita deve essere minore della sua capacità

\begin{equation}
u_{i} \leq Q quad \; \; \forall \;i \; \in \; V
\tag{4}
\end{equation}

-  Vincolo di capacità e di assenza di sottogiri

\begin{equation}
u_{i} - u_{j} \geq d_{j}*x_{ij} - Q*(1 - x_{ij}) \quad \; \; \;i,j \; \in \; V, \;i \neq j, \;j \neq 1
\tag{4}
\end{equation}

- Se un camion $k$ attraversa l'arco $(i,j)$ allora servirò $j$ dopo aver servito il cliente $i$

\begin{equation}
a_{j} \geq (a_{i} + t_{i} + t_{ij}) - (1 - x_{ij}) * T \quad \; \; \; i,j \in V
\end{equation}

- Se arrivo in j nella sua finestra temporale lo servo subito

\begin{equation}
a_{j} \leq (a_{i} + t_{i} + t_{ij}) + (1 - x_{ij}) * T \quad \; \; \; i,j \in V
\end{equation}

- L'orario di inizio per servire il nodo $i$ deve essere incluso nella finestra temporale del nodo

\begin{equation}
l_{j} \leq a_{j} \leq L_{j} 
\end{equation}

In [ ]:
mod.addConstrs((gb.quicksum(x.select(i, '*')) - gb.quicksum(x.select('*', i)) == 0 for i in range(problem["nodes_num"])), "Trasporto")
mod.addConstr(gb.quicksum(x.select(0, '*')) <= problem["num_veh"], "NumeroVeicoli")
mod.addConstrs((gb.quicksum(x.select(i, '*')) == 1 for i in range(1,problem["nodes_num"])), "VeicoliCliente")
mod.addConstr((u[0] == 0), "DomandaIniziale")
mod.addConstrs((u[i] <= problem["capacity"] for i in range(problem["nodes_num"])), "CapacitàMassima")
mod.addConstrs((u[j] - u[i] >= problem["nodes"][j]["demand"] * x[i, j] - problem["capacity"] * (1 - x[i, j]) for i in range(problem["nodes_num"]) for j in range(problem["nodes_num"]) if i != j and j != 0), "Sottogiri&capacità")
mod.addConstrs((x[i, i] == 0 for i in range(problem["nodes_num"])), "ArchiRicorsivi")
mod.addConstrs((a[j] >= (a[i] + problem["nodes"][i]["service_time"] + problem["t_matrix"][i,j]) - 1000000 * (1 - x[i, j]) for i in range(problem["nodes_num"]) for j in range(1, problem["nodes_num"]) if i != j), "VincoloTempo1")
mod.addConstrs((a[j] <= (a[i] + problem["nodes"][i]["service_time"] + problem["t_matrix"][i,j]) + 1000000 + (1 - x[i,j]) for i in range(problem["nodes_num"]) for j in range(problem["nodes_num"]) if i != j), "VincoloTempo2")
mod.addConstrs((a[j] <= problem["nodes"][j]["due_date"] for j in range(problem["nodes_num"])), "FinestreTemporali1")
mod.addConstrs((a[j] >= problem["nodes"][j]["rdy_time"] for j in range(problem["nodes_num"])), "FinestreTemporali2")
#mod.addConstr((a[0] == 0), "FinestreTemporali4")
#mod.addConstrs(((a[i] + problem["nodes"][i]["service_time"] + problem["t_matrix"][i, 0]) <= 10000 * (1 - x[i, 0]) + problem["nodes"][0]["due_date"] * x[i, 0] for i in range(1, problem["nodes_num"])), "FinestreTemporali5")

In [ ]:
obj = (gb.quicksum(problem["t_matrix"][i, j] * x[i, j] for i in range(problem["nodes_num"]) for j in range(problem["nodes_num"]) if i != j))
mod.setObjective(obj, GRB.MINIMIZE)
mod.optimize()